# Mini lab - Building a local Knowledge Graph RAG with Neo4j, LangChain, and Ollama 🦙




## 1. Setup and Installation


In [61]:
import os
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import DiffbotGraphTransformer
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_neo4j import GraphCypherQAChain
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv


## 2. Configuration


In [62]:
#importing diffbot api key from .env file
load_dotenv()
diffbot_api_key= os.getenv("DIFFBOT_API_KEY")
# neo4j parameters to enable the connection to the database
NEO4J_URI ="bolt://localhost:7687"
NEO4J_USERNAME ="neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "shop"  
#ollama model
OLLAMA_MODEL = "llama3:8b"  

# person to search
TOPIC= "Satoshi Nakamoto" 


## 3. Load Wikipedia article


In [63]:

print(f"Loading Wikipedia article about: {TOPIC}")
loader = WikipediaLoader(query=TOPIC,load_max_docs=1)
documents = loader.load()

print(f"Loaded {len(documents)} document(s)")
print(f"Document length: {len(documents[0].page_content)} characters")
print(f"\nFirst 500 characters:\n{documents[0].page_content[:500]}...")


Loading Wikipedia article about: Satoshi Nakamoto
Loaded 1 document(s)
Document length: 4000 characters

First 500 characters:
Satoshi Nakamoto  (fl. 31 October 2008 – 26 April 2011) is the name used by the presumed pseudonymous person or persons who developed bitcoin, authored the bitcoin white paper, and created and deployed bitcoin's original reference implementation. As part of the implementation, Nakamoto also devised the first blockchain database. Nakamoto was active in the development of bitcoin until December 2010.
Nakamoto's true identity is unknown, although various people have been posited as the person or gr...


In [64]:
# Clean the database before ingesting new data
from langchain_community.graphs import Neo4jGraph

database = NEO4J_DATABASE if 'NEO4J_DATABASE' in globals() else "shop"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=database
)

graph.query("MATCH (n) DETACH DELETE n")
print("Database cleaned")


Database cleaned


## 4. Extract entities and relationships with Diffbot


In [65]:


graph_transformer = DiffbotGraphTransformer(diffbot_api_key=os.getenv("DIFFBOT_API_KEY"))
graph_documents = graph_transformer.convert_to_graph_documents(documents)

# display results
sample = graph_documents[0] if graph_documents else None
print(f"Extracted {len(graph_documents)} graph document")
if sample:
    print(f"Nodes: {len(sample.nodes)}, relationships: {len(sample.relationships)}")
    if sample.nodes:
        print(f"Sample node:{sample.nodes[0]}")
    if sample.relationships:
        print(f"Sample relationship: {sample.relationships[0]}")


Extracted 1 graph document
Nodes: 7, relationships: 9
Sample node:id='http://www.wikidata.org/entity/Q13382352' type='Person' properties={'name': 'Satoshi Nakamoto', 'dateOfDeath': '2011-04-26', 'dateOfBirth': '2008-10-31'}
Sample relationship: source=Node(id='Mike Hearn', type='Person', properties={}) target=Node(id='http://www.wikidata.org/entity/Q13382352', type='Person', properties={}) type='WORK_RELATIONSHIP' properties={'evidence': 'In 2011, Nakamoto wrote in an email to co-developer Mike Hearn that he had “moved on to other things,” and he was never heard from again.'}


## 5. Connect to Neo4j and Ingest Graph

```bash
docker compose up -d
```
check on http://localhost:7474 for neo4j


In [66]:
# Connect to Neo4j
database =NEO4J_DATABASE if 'NEO4J_DATABASE' in globals() else "shop"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=database
)

print("Connected to Neo4j")




Connected to Neo4j


In [67]:
# Add graph documents to Neo4j
print("Ingesting graph documents into Neo4j...")
graph.add_graph_documents(graph_documents)
print("Graph ingestion completed")


Ingesting graph documents into Neo4j...
Graph ingestion completed


## 6. Graph Exploration and Analysis


In [68]:
# list all node labels
labels_query = "CALL db.labels()"
labels = graph.query(labels_query)
print("Node labels in the graph:")
for label in labels:
    print(f" - {label.get('label', 'N/A')}")


Node labels in the graph:
 - Person
 - Location
 - Skill
 - Organization


In [69]:
# inspect all relationship
relationship_types_query ="CALL db.relationshipTypes()"
relationship_types =graph.query(relationship_types_query)
print("relationship types in the graph:")
for rel_type in relationship_types:
    print(f"-{rel_type.get('relationshipType', 'N/A')}")


relationship types in the graph:
-WORK_RELATIONSHIP
-PERSON_LOCATION
-INTERESTED_IN
-EMPLOYEE_OR_MEMBER_OF


In [70]:
# find the target entity for satoshi nakamoto as we took this example
entity_query = f"""
MATCH (p:Person {{name: '{TOPIC}'}})
RETURN p
LIMIT 5
"""

entities = graph.query(entity_query)
print(f"Found {len(entities)} entity(ies) matching '{TOPIC}':")
for entity in entities:
    print(entity)


Found 1 entity(ies) matching 'Satoshi Nakamoto':
{'p': {'dateOfDeath': '2011-04-26', 'name': 'Satoshi Nakamoto', 'dateOfBirth': '2008-10-31', 'id': 'http://www.wikidata.org/entity/Q13382352'}}


In [71]:
#query relationships around the target entity
relationships_query = f"""
MATCH (p:Person)-[r]->(related)
WHERE p.name CONTAINS '{TOPIC.split()[0]}' OR p.name CONTAINS '{TOPIC.split()[-1] if len(TOPIC.split()) > 1 else TOPIC}'
RETURN p.name as person, type(r) as relationship, related.name as related_entity, labels(related) as related_type
LIMIT 20
"""

relationships =graph.query(relationships_query)
print(f"relationships for entities related to'{TOPIC}':")
for rel in relationships :
    print(f"{rel['person']} --[{rel['relationship']}]-->{rel['related_entity']} ({rel['related_type']})")


relationships for entities related to'Satoshi Nakamoto':
Satoshi Nakamoto --[EMPLOYEE_OR_MEMBER_OF]-->P2P Foundation (['Organization'])
Satoshi Nakamoto --[WORK_RELATIONSHIP]-->Mike Hearn (['Person'])
Satoshi Nakamoto --[PERSON_LOCATION]-->Japan (['Location'])
Satoshi Nakamoto --[INTERESTED_IN]-->software (['Skill'])


## 7. Graph Exploration Queries


In [72]:
# most connected people
most_connected_query = """
MATCH (p:Person)-[r]-()
RETURN p.name as person, count(r) as connections
ORDER BY connections DESC
LIMIT 10
 """

results = graph.query(most_connected_query)
print("most connected people:")
for result in results:
    print(f"  {result['person']}:{result['connections']} connections")


most connected people:
  Satoshi Nakamoto:5 connections
  Mike Hearn:2 connections
  Laszlo Hanyecz:2 connections
  Dan Kaminsky:2 connections


In [73]:
# employment relationships
employment_query = """
MATCH (p:Person)-[r:EMPLOYEE_OR_MEMBER_OF|WORK_RELATIONSHIP]->(o)
RETURN p.name as person, o.name as organization, type(r) as relationship
LIMIT 20
  """

results = graph.query(employment_query)
print("Employment relationships: ")
for result in results:
    print(f"  {result['person']}-[{result['relationship']}]-->{result['organization']}")


Employment relationships: 
  Satoshi Nakamoto-[WORK_RELATIONSHIP]-->Mike Hearn
  Satoshi Nakamoto-[EMPLOYEE_OR_MEMBER_OF]-->P2P Foundation
  Mike Hearn-[WORK_RELATIONSHIP]-->Satoshi Nakamoto
  Laszlo Hanyecz-[WORK_RELATIONSHIP]-->Dan Kaminsky
  Dan Kaminsky-[WORK_RELATIONSHIP]-->Laszlo Hanyecz


In [74]:
#locations
locations_query = """
MATCH (p:Person)-[r]->(l)
WHERE 'Location' IN labels(l) OR 'Place' IN labels(l)
RETURN p.name as person, l.name as location, type(r) as relationship
LIMIT 20
"""
results =graph.query(locations_query)
print("Location relationships:")
for result in results:
    print(f" {result['person']} -[{result['relationship']}]-->{result['location']} ")


Location relationships:
 Satoshi Nakamoto -[PERSON_LOCATION]-->Japan 


## Verify Target Entity Exists

In [75]:

# only check if the entity exists
entity_query = f"""
MATCH (p:Person {{name: '{TOPIC}'}})
RETURN p, id(p) as node_id
LIMIT 10
"""

entities = graph.query(entity_query)
print(f"Found {len(entities)} entity(ies) matching '{TOPIC}':\n")

for i, entity in enumerate(entities, 1):
    person =entity.get('p', {})
    node_id = entity.get('node_id')
    print(f"Entity {i} (Node ID: {node_id}): ")
    print(f"  Properties: {person}")

# multiple entities
if len(entities) >1:
    print("Note: If multiple entities exist, choose by:")
    print("  - Node ID (most unique) or addidtional properties such as id or dates or something else")
elif len(entities) ==0:
    print("No entity found with that exact name ")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 11, offset: 55} for query: "\nMATCH (p:Person {name: 'Satoshi Nakamoto'})\nRETURN p, id(p) as node_id\nLIMIT 10\n"


Found 1 entity(ies) matching 'Satoshi Nakamoto':

Entity 1 (Node ID: 0): 
  Properties: {'dateOfDeath': '2011-04-26', 'name': 'Satoshi Nakamoto', 'dateOfBirth': '2008-10-31', 'id': 'http://www.wikidata.org/entity/Q13382352'}


## 8. Reusable Function to Query Relationships


In [76]:
def get_all_relationships_for_person(graph, person_name):

    query = """
    MATCH (p:Person)-[r]-(related)
    WHERE p.name CONTAINS $person_name
    RETURN p.name as person, 
           type(r) as relationship_type, 
           related.name as related_entity, 
           labels(related) as related_labels,
           startNode(r) = p as outgoing
    ORDER BY relationship_type
    """
    
    results = graph.query(query,params={"person_name":person_name})
    return results

# test 
person_relationships =get_all_relationships_for_person(graph, TOPIC.split()[0])
print(f"All relationships for '{TOPIC.split()[0]}':")
for rel in person_relationships[:10]: 
    direction = "--[%s]-->" if rel['outgoing'] else "<--[%s]--"
    print(f"{rel['person']} {direction % rel['relationship_type']} {rel['related_entity']} {rel['related_labels']}")


All relationships for 'Satoshi':
 Satoshi Nakamoto --[EMPLOYEE_OR_MEMBER_OF]--> P2P Foundation ['Organization']
 Satoshi Nakamoto --[INTERESTED_IN]--> software ['Skill']
 Satoshi Nakamoto --[PERSON_LOCATION]--> Japan ['Location']
 Satoshi Nakamoto --[WORK_RELATIONSHIP]--> Mike Hearn ['Person']
 Satoshi Nakamoto <--[WORK_RELATIONSHIP]-- Mike Hearn ['Person']


## 9. Create GraphCypherQAChain with Ollama


### Check Available Relationship Types

Before creating the chain, let's verify what relationship types actually exist in the database:


In [77]:
# Check what relationship types actually exist
relationship_types_query = "CALL db.relationshipTypes()"
relationship_types = graph.query(relationship_types_query)
print("Available relationship types in the database:")
for rel_type in relationship_types:
    print(f"  - {rel_type.get('relationshipType', 'N/A')}")

# Check node labels
labels_query = "CALL db.labels()"
labels = graph.query(labels_query)
print("\nAvailable node labels:")
for label in labels:
    print(f"  - {label.get('label', 'N/A')}")


Available relationship types in the database:
  - WORK_RELATIONSHIP
  - PERSON_LOCATION
  - INTERESTED_IN
  - EMPLOYEE_OR_MEMBER_OF

Available node labels:
  - Person
  - Location
  - Skill
  - Organization


In [78]:

# initialize chat model
llm = ChatOllama(
    model=OLLAMA_MODEL,
    temperature=0 # so that the model is as cartesian as possible, not using any creativity
)

# cypher generation prompt
CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.

CRITICAL RULES - READ CAREFULLY:
1. Use ONLY the exact relationship types listed in the schema below - DO NOT invent new ones
2. Use ONLY the exact node labels from the schema (e.g., "Person", not "SatoshiNakamoto")
3. Common relationship types in this database: EMPLOYEE_OR_MEMBER_OF, WORK_RELATIONSHIP, PERSON_LOCATION, INTERESTED_IN
4. For "associated with" or "organizations" queries, use: EMPLOYEE_OR_MEMBER_OF or WORK_RELATIONSHIP
5. If a relationship type is not in the schema, use a generic pattern: MATCH (p:Person)-[r]-(related) WHERE p.name = "..." RETURN related
6. NEVER use relationship types like: ASSOCIATED_WITH, CREATED, PUBLISHED, RELATED_TO - these do NOT exist

Schema:
{schema}

MANDATORY INSTRUCTIONS:
- Do not include any explanations or apologies
- Do not respond to questions that ask for anything other than a Cypher statement
- Do not include any text except the generated Cypher statement
- Use ONLY relationship types that appear EXACTLY in the schema above
- Use ONLY node labels that appear EXACTLY in the schema above

The question is:
{question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template=CYPHER_GENERATION_TEMPLATE
)

# Create the GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    verbose=True,
    allow_dangerous_requests=True
)

print("GraphCypherQAChain created successfully!")


GraphCypherQAChain created successfully!


### Improved Query Function

The chain is generating correct Cypher but not interpreting results well. Let's create a better query function:


In [80]:
# Improved query function that includes relationship types for better answers
def ask_graph_improved(question):
    """
    Ask a question with better Cypher query that includes relationship types.
    """
    try:
        # First, try the chain
        response = chain.invoke({"query": question})
        result = response.get("result", "")
        
        # If result is empty or says "I don't know", try a more direct query
        if not result or "don't know" in result.lower() or "I don't know" in result:
            # For relationship questions, use a query that includes relationship types
            if "relationship" in question.lower() or "connected" in question.lower() or "related" in question.lower():
                direct_query = f"""
                MATCH (p:Person {{name: '{TOPIC}'}})-[r]-(related)
                RETURN p.name as person, type(r) as relationship, related.name as related_entity, labels(related) as related_type
                ORDER BY relationship
                """
                results = graph.query(direct_query)
                if results:
                    answer = f"The main relationships for {TOPIC} are:\n"
                    for rel in results:
                        answer += f"  - {rel['relationship']}: {rel['related_entity']} ({', '.join(rel['related_type'])})\n"
                    return answer
        
        return result
    except Exception as e:
        return f"Error: {str(e)}"

# Test the improved function
test_question = f"What are the main relationships connected to {TOPIC}?"
print(f"Q: {test_question}")
answer = ask_graph_improved(test_question)
print(f"A: {answer}")


Q: What are the main relationships connected to Satoshi Nakamoto?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name:"Satoshi Nakamoto"})-[r]-(related) RETURN related;
Full Context:
[{'related': {'name': 'software', 'id': 'http://www.wikidata.org/entity/Q7397'}}, {'related': {'name': 'Japan', 'id': 'http://www.wikidata.org/entity/Q17'}}, {'related': {'name': 'Mike Hearn', 'id': 'Mike Hearn'}}, {'related': {'name': 'P2P Foundation', 'id': 'P2P Foundation'}}, {'related': {'name': 'Mike Hearn', 'id': 'Mike Hearn'}}]

> Finished chain.
A: The main relationships for Satoshi Nakamoto are:
  - EMPLOYEE_OR_MEMBER_OF: P2P Foundation (Organization)
  - INTERESTED_IN: software (Skill)
  - PERSON_LOCATION: Japan (Location)
  - WORK_RELATIONSHIP: Mike Hearn (Person)
  - WORK_RELATIONSHIP: Mike Hearn (Person)



In [82]:
# helper function to ask questions to the graph
def ask_graph(question):
    """
    Ask a natural language question to the knowledge graph.
    
    Args:
        question: Natural language question about the graph
    
    Returns:
        Answer from the LLM based on graph data
    """
    try:
        response = chain.invoke({"query": question})
        return response["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# some Q
questions = [
    f"Who is {TOPIC}?",
    f"What organizations is {TOPIC} associated with?",
]

for question in questions:
    print(f"\nQ: {question}")
    answer = ask_graph(question)
    print(f"A: {answer}")



Q: Who is Satoshi Nakamoto?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[r:EMPLOYEE_OR_MEMBER_OF|WORK_RELATIONSHIP]-(related) WHERE p.name = "Satoshi Nakamoto" RETURN related;
Full Context:
[{'related': {'name': 'Mike Hearn', 'id': 'Mike Hearn'}}, {'related': {'name': 'P2P Foundation', 'id': 'P2P Foundation'}}, {'related': {'name': 'Mike Hearn', 'id': 'Mike Hearn'}}]

> Finished chain.
A: Satoshi Nakamoto.

Q: What organizations is Satoshi Nakamoto associated with?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name:"Satoshi Nakamoto"})-[:EMPLOYEE_OR_MEMBER_OF|WORK_RELATIONSHIP]-(o) RETURN o;
Full Context:
[{'o': {'name': 'Mike Hearn', 'id': 'Mike Hearn'}}, {'o': {'name': 'P2P Foundation', 'id': 'P2P Foundation'}}, {'o': {'name': 'Mike Hearn', 'id': 'Mike Hearn'}}]

> Finished chain.
A: Mike Hearn, P2P Foundation.


In [83]:
def get_entity_subgraph(graph, entity_name, max_depth=2):
    """
    Get a subgraph around a specific entity.
    """
    query = f"""
    MATCH path = (start)-[*1..{max_depth}]-(connected)
    WHERE start.name CONTAINS $entity_name
    WITH path, start, connected
    RETURN DISTINCT
        start.name as start_node,
        labels(start) as start_labels,
        connected.name as connected_node,
        labels(connected) as connected_labels,
        [rel in relationships(path) | type(rel)] as relationship_types
    LIMIT 50
    """
    
    results = graph.query(query, params={"entity_name": entity_name})
    return results

# get subgraph for the topic
subgraph = get_entity_subgraph(graph, TOPIC.split()[0])
print(f"Subgraph for '{TOPIC}':")
print(f"Found {len(subgraph)} connections")
for i, connection in enumerate(subgraph[:5]):  # Show first 5
    print(f"  {i+1}. {connection['start_node']} -> {connection['connected_node']} via {connection['relationship_types']}")


Subgraph for 'Satoshi Nakamoto':
Found 5 connections
  1. Satoshi Nakamoto -> Mike Hearn via ['WORK_RELATIONSHIP']
  2. Satoshi Nakamoto -> Satoshi Nakamoto via ['WORK_RELATIONSHIP', 'WORK_RELATIONSHIP']
  3. Satoshi Nakamoto -> P2P Foundation via ['EMPLOYEE_OR_MEMBER_OF']
  4. Satoshi Nakamoto -> Japan via ['PERSON_LOCATION']
  5. Satoshi Nakamoto -> software via ['INTERESTED_IN']


In [84]:
def summarize_entity(graph, llm, entity_name):
    """
    Generate a summary of an entity based on its graph context.
    
    """
    # Get entity information
    entity_info_query = """
    MATCH (e)
    WHERE e.name CONTAINS $entity_name
    OPTIONAL MATCH (e)-[r]-(related)
    RETURN e.name as name,
           labels(e) as labels,
           collect(DISTINCT {type: type(r), related: related.name, related_labels: labels(related)}) as relationships
    LIMIT 1
    """
    entity_data = graph.query(entity_info_query, params={"entity_name": entity_name})
    
    if not entity_data:
        return f"No information found about {entity_name}"
    
    data = entity_data[0]
    
    # build context for LLM
    context = f"""
    Entity: {data['name']}
    Labels: {', '.join(data['labels'])}
    
    Relationships:
    """
    
    for rel in data['relationships'][:10]:  # Limit to first 10 relationships
        context += f"  - {rel['type']} -> {rel['related']} ({', '.join(rel['related_labels'])})\n"
    
    # Generate summary
    prompt = f"""Based on the following graph information, provide a concise summary of {data['name']}.
    Only use information explicitly present in the graph data. Do not add any information that is not in the provided data.
    
    {context}
    
    Summary:"""
    
    response = llm.invoke(prompt)
    return response.content

# generate summary
print(f"Generating summary for '{TOPIC}'...")
summary = summarize_entity(graph, llm, TOPIC.split()[0])
print(f"\nSummary:\n{summary}")


Generating summary for 'Satoshi Nakamoto'...

Summary:
Based on the graph data, Satoshi Nakamoto is a person who:

* Has an interest in software as a skill
* Is located in Japan
* Has a work relationship with Mike Hearn
* Is an employee or member of the P2P Foundation organization


## 11. Additional Exploration


## Questions

1. **What elements in the custom prompt help the model generate correct Cypher?** <br>
<br/>The schema information, clear instructions, the note about not including explanation


2. **What information from the graph would you add/remove to improve summary quality?**<br>
 <br/>  I would add d ates or locations for more factual summaries and relationships directions and weights and I would delete generic relationships that don't add context in the case of the person
   Just asking the LLM what I exactly want and need, with exact structure as it can hallucinate sometimes

3. **Does the LLM hallucinate dates or facts? How would you constrain it?**<br>
<br/>Yes, LLMs can hallucinate. To constrain we can use structured queries to extract specific fact, limit the model's temperature to reduce creativity or add explicit instructions to only use provided data
